In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re


In [115]:
path='//192.168.14.50/Public/ВЫГРУЗКИ/'
files=os.listdir(path)


In [116]:

def shop_name(shop):

    m1 = re.compile('([0-9]{2}-[0-9]{,2}-)([0-9]{,3})')
    m = m1.findall(shop)
    for i in range(len(m)):
        m[i] = m[i][0] + ('000' + m[i][1])[-3:]
    return m[0]

In [117]:

ds=pd.read_csv(path + 'ПРОДАЖИ ПО ДНЯМ (таблица для анализа) от 04-09-24.txt', sep='\t')
ds['Выручка,']=ds['Выручка,'].str.replace(',','.')
ds['Выручка,']=ds['Выручка,'].str.replace(chr(160),'')
ds['Количество']=ds['Количество'].str.replace(',','.')
ds['Выручка,']=pd.to_numeric(ds['Выручка,'])
ds['День.Начало дня']=pd.to_datetime(ds['День.Начало дня'],dayfirst=True,errors='coerce')
ds=ds[ds['День.Начало дня'].isna()==False]
ds=ds.assign(Номер_тт= ds['Склад / магазин'].apply(shop_name))
ds[['key','product']]=ds['Код, Номенклатура'].str.split(',',n=1,expand=True)
ds['Выручка,'].fillna(0)
ds['Количество']=pd.to_numeric(ds['Количество'])
ds= ds.dropna(subset=['Количество'])
ds=ds.drop(['Код, Номенклатура'],axis=1)
ds['weekday']= pd.DatetimeIndex(ds['День.Начало дня']).weekday
ds['day']= pd.DatetimeIndex(ds['День.Начало дня']).day
ds['month']= pd.DatetimeIndex(ds['День.Начало дня']).month
ds['year']= pd.DatetimeIndex(ds['День.Начало дня']).year


In [118]:
ds.columns=['date', 'shop_name', 'person', 'feature',
       'amount', 'sales', 'shop_id', 'key', 'product','weekday','day','month', 'year'       ]

In [ ]:
ds.describe(include='all')

In [120]:
ds_prod=ds[['product','key','feature']].drop_duplicates().reset_index(drop=True)
ds_shops=ds[['shop_name']].drop_duplicates().reset_index(drop=True)
ds=ds.reset_index(drop=True)

In [121]:
daily_per_shop=ds.groupby(['shop_id','weekday','day','month','year'],as_index=False).agg({'key':['count'],'amount':['sum','mean','median','std'],'sales':['sum','mean','median','std']})
daily_per_shop.columns=[ '_'.join(x) for x in daily_per_shop.columns]


In [ ]:
daily_per_shop